In [63]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np 
import pickle 
import os

os.makedirs('modelo', exist_ok=True)

df = pd.read_csv('data/datos.csv')
df_1 = pd.read_csv('webscrapping/tesla_autoscout24.csv')
df_1

,Modelo,Precio,Color,Pais,Kilometraje,date
0,Model 3,20890.0,Azul,España,143000,2019.0
1,Model 3,27500.0,Azul,España,39000,2019.0
2,Model 3,25850.0,Azul,España,93000,2021.0
3,Model 3,24900.0,Azul,España,128000,2019.0
4,Model 3,28990.0,Azul,España,39413,2019.0
...,...,...,...,...,...,...
1335,Model Y,52000.0,Gris,Italia,18000,2024.0
1336,Model Y,42500.0,Gris,Italia,42000,2022.0
1337,Model Y,38000.0,Gris,Italia,32000,2023.0
1338,Model Y,53400.0,Gris,Italia,32000,2023.0


In [64]:
df_1.dropna(inplace=True)

In [65]:
df_1 = df_1[['Modelo', 'Precio','date','Kilometraje', 'Color', 'Pais']]
df_1.rename(columns={'Modelo' : 'model', 'Precio' : 'price', 'Color' : 'color', 'Pais': 'country', 'Kilometraje' : 'km', 'date' : 'year' }, inplace=True)
df_1['year'] = df_1['year'].astype('int')
df_1

,model,price,year,km,color,country
0,Model 3,20890.0,2019,143000,Azul,España
1,Model 3,27500.0,2019,39000,Azul,España
2,Model 3,25850.0,2021,93000,Azul,España
3,Model 3,24900.0,2019,128000,Azul,España
4,Model 3,28990.0,2019,39413,Azul,España
...,...,...,...,...,...,...
1335,Model Y,52000.0,2024,18000,Gris,Italia
1336,Model Y,42500.0,2022,42000,Gris,Italia
1337,Model Y,38000.0,2023,32000,Gris,Italia
1338,Model Y,53400.0,2023,32000,Gris,Italia


In [66]:
df = pd.concat([df, df_1])
df.drop_duplicates(inplace=True)
df.to_csv('data/datos_limpios_finales.csv', index=False)

Preparación de Datos

In [67]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df[['km']])
with open('modelo/scaler_km.pkl', 'wb') as f:
    pickle.dump(scaler, f)
df['km'] = scaler.transform(df[['km']])

In [68]:
le_model = LabelEncoder()
le_model.fit(df[['model']])
with open('modelo/le_model.pkl', 'wb') as f:
    pickle.dump(le_model, f)
df['model'] = le_model.transform(df[['model']])

le_color = LabelEncoder()
le_color.fit(df[['color']])
with open('modelo/le_color.pkl', 'wb') as f:
    pickle.dump(le_color, f)
df['color'] = le_color.transform(df[['color']])

le_year = LabelEncoder()
le_year.fit(df[['year']])
with open('modelo/le_year.pkl', 'wb') as f:
    pickle.dump(le_year, f)
df['year'] = le_year.transform(df[['year']])

le_country = LabelEncoder()
le_country.fit(df[['country']])
with open('modelo/le_country.pkl', 'wb') as f:
    pickle.dump(le_country, f)
df['country'] = le_country.transform(df[['country']])

df

/home/kike/miniconda3/envs/.env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kike/miniconda3/envs/.env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/kike/miniconda3/envs/.env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kike/miniconda3/envs/.env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWa

,model,price,year,km,color,country
0,1,36999.0,6,0.995464,1,0
1,1,59900.0,4,-0.011106,1,0
4,1,84900.0,10,-0.732482,1,0
5,1,89900.0,10,-0.682154,1,0
6,1,55900.0,7,-0.078211,1,0
...,...,...,...,...,...,...
1335,3,52000.0,11,-0.648601,3,2
1336,3,42500.0,9,-0.245973,3,2
1337,3,38000.0,10,-0.413735,3,2
1338,3,53400.0,10,-0.413735,3,2


In [69]:
from sklearn.model_selection import train_test_split

X= df.drop(columns=['price'])
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [70]:
model = RandomForestRegressor(n_estimators=5000, random_state=137)

model.fit(X_train, y_train)
with open('modelo/modelo_entrenado.pkl', 'wb') as f:
    pickle.dump(model, f)
y_pred = model.predict(X_test)

mse = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mse:.2f}")
print(f"R2 Score: {r2:.2f}")

Mean Absolute Error: 4909.02
R2 Score: 0.88
